In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()  # Loads variables from .env into environment

from google.adk.agents import LlmAgent, SequentialAgent
from google.adk.runners import Runner
from google.adk.models.google_llm import Gemini
from google.adk.tools import google_search, preload_memory
from google.genai import types
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService

print("✅ ADK components imported successfully.")

In [2]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if GOOGLE_API_KEY:
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
else:
    print("🔑 Authentication Error: 'GOOGLE_API_KEY' not found in .env file.")

✅ Gemini API key setup complete.


In [3]:
async def run_session(
    runner_instance: Runner, user_queries: list[str] | str, session_id: str = "default"
):
    """Helper function to run queries in a session and display responses."""
    print(f"\n### Session: {session_id}")

    # Create or retrieve session
    try:
        session = await session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )
    except:
        session = await session_service.get_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )

    # Convert single query to list
    if isinstance(user_queries, str):
        user_queries = [user_queries]

    # Process each query
    for query in user_queries:
        print(f"\nUser > {query}")
        query_content = types.Content(role="user", parts=[types.Part(text=query)])

        # Stream agent response
        async for event in runner_instance.run_async(
            user_id=USER_ID, session_id=session.id, new_message=query_content
        ):
            if event.is_final_response() and event.content and event.content.parts:
                text = event.content.parts[0].text
                if text and text != "None":
                    print(f"Model: > {text}")


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [4]:
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
    last_agent_only: bool = False,
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK Content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            responses = []  # Buffer to collect responses with agent names

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        # Get agent name from author attribute
                        agent_name = getattr(event, 'author', 'Unknown Agent')

                        if last_agent_only:
                            # Collect all responses with agent names
                            responses.append({
                                'agent': agent_name,
                                'text': event.content.parts[0].text
                            })
                        else:
                            # Print all responses
                            print(f"{agent_name} > ", event.content.parts[0].text)

            # Print only the last response if filtering
            if last_agent_only and responses:
                last_response = responses[-1]
                print(f"{last_response['agent']} > ", last_response['text'])
    else:
        print("No queries!")

In [5]:
async def auto_save_to_memory(callback_context):
    """Automatically save session to memory after each agent turn."""
    try:
        agent_name = callback_context._invocation_context.agent.name
        session = callback_context._invocation_context.session

        await callback_context._invocation_context.memory_service.add_session_to_memory(session)

        print(f"💾 Saved {agent_name} output to memory")
    except Exception as e:
        print(f"⚠️  Warning: Failed to save to memory: {e}")
        # Don't raise - let the agent continue even if memory save fails


print("✅ Callback created.")

✅ Callback created.


In [6]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [ ]:
# 1. Research Agent
research_agent = LlmAgent(
    name="research_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    You are a Research Agent specialized in career transitions.
    
    Your task:
    1. Extract career transition details from the user's message or check preload_memory for context
    2. Use the Google Search tool to research the target career path
    3. Focus on: required skills, typical career progression, salary ranges, and job market demand
    4. Search for: online courses, certifications, and learning resources
    5. Look for: success stories of people who made similar transitions
    
    Output format:
    ## Key Skills Required
    [List 5-7 essential skills with brief descriptions]
    
    ## Learning Resources
    [Specific courses, certifications, books, and platforms]
    
    ## Market Outlook
    [Job demand, salary ranges, growth trends with data]
    
    ## Transition Timeline
    [Typical timeframe for this transition based on the user's experience level]
    
    ## Success Stories
    [Brief examples of successful transitions]
    
    Keep your research comprehensive but concise. Focus on actionable information tailored to the user's background.
    """,
    tools=[google_search, preload_memory],
    output_key="research_summary", 
    after_agent_callback=auto_save_to_memory,
)

In [ ]:
# 2. Mentor Agent
mentor_agent = LlmAgent(
    name="mentor_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    You are a Career Mentor Agent. Based on the research findings: {research_summary}
    
    Extract the user's context from the conversation history and research summary, then create a personalized, actionable transition plan.
    
    Structure your plan as follows:
    
    ## Phase 1: Foundation (Months 1-3)
    - Specific skills to learn first (prioritized based on their background)
    - Recommended courses/resources with links when available
    - Daily/weekly time commitment suggestions
    - Quick wins to build confidence
    
    ## Phase 2: Building Portfolio (Months 4-6)
    - Concrete projects to build (with examples relevant to their experience)
    - GitHub repositories to create
    - Communities to join (specific names)
    - Networking strategies leveraging their current role
    
    ## Phase 3: Job Search (Months 6-9)
    - Resume updates needed (specific sections)
    - Where to apply (companies, job boards)
    - Interview preparation tips
    - Portfolio presentation strategies
    
    ## Milestones & Checkpoints
    - Monthly goals with measurable outcomes
    - How to measure progress
    - Red flags and when to adjust course
    
    ## Leveraging Your Background
    - How to translate their existing skills to the new role
    - Unique advantages from their current position
    - How their experience is an asset
    
    Make it specific, realistic, and encouraging. Use actual course names, platforms, and communities when possible.
    Keep the tone supportive but practical - acknowledge challenges while emphasizing achievability.
    Tailor everything to their specific situation.
    """,
    output_key="career_advice",
)

In [ ]:
# 3. Root Agent to orchestrate the workflow
root_agent = SequentialAgent(
    name="CareerPathPipeline",
    sub_agents=[research_agent, mentor_agent],
)

print("✅ Root agent created with 2-agent pipeline: Research -> Mentor")

In [ ]:
APP_NAME = "default"  # Application
USER_ID = "default"  # User
SESSION = "default"  # Session

MODEL_NAME = "gemini-2.5-flash-lite"

# Step 2: Set up Session Management
# InMemorySessionService stores conversations in RAM (temporary)
session_service = InMemorySessionService()
memory_service = InMemoryMemoryService()  # ADK's built-in Memory Service for development and testing

# Step 3: Create the Runner
runner = Runner(
    agent=root_agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service,
)

print("✅ Stateful agent initialized!")
print(f"   - Application: {APP_NAME}")
print(f"   - User: {USER_ID}")
print(f"   - Using: {session_service.__class__.__name__}")

In [ ]:
# Testing - Provide career transition context
await run_session(
    runner,
    "I am a Product Manager with 5 years of experience. I want to transition to data science and can dedicate 10 hours per week.",
    session_name="career-transition-test",
    last_agent_only=True,
)

In [14]:
# Start a conversation
await run_session(runner, "I want to change careers", "my-session")

await run_session(runner, "Product Manager", "my-session")
# Intake Agent: "Great! How many years of experience do you have as a Product Manager?"

await run_session(runner, "5 years", "my-session")
# Intake Agent: "Perfect! What role are you looking to transition to?"

await run_session(runner, "Data Scientist", "my-session")
# Intake Agent: "Excellent choice! How many hours per week can you dedicate to learning?"

await run_session(runner, "10 hours", "my-session")
# Intake Agent: [Calls validate_user_profile()] ✅ Profile complete!
# Research Agent: [Researches Data Science transition]
# Mentor Agent: [Creates personalized plan]
# Intake Agent: "I'd be happy to help! What is your current role?"


 ### Session: my-session

User > I want to change careers
intake_agent >  Yes, I see that in our previous conversation. To get started, what is your current role?

💾 Saved intake_agent output to memory
research_agent >  I understand you're looking to change careers. To help me provide the most relevant information, could you please tell me your current role? Once I have that, I can research potential new career paths for you, including the skills, resources, and market outlook for those fields.
💾 Saved research_agent output to memory


KeyError: 'Context variable not found: `time_commitment_hours`.'

In [ ]:

await run_session(runner, "Product Manager", "my-session")
# Intake Agent: "Great! How many years of experience do you have as a Product Manager?"

await run_session(runner, "5 years", "my-session")
# Intake Agent: "Perfect! What role are you looking to transition to?"

await run_session(runner, "Data Scientist", "my-session")
# Intake Agent: "Excellent choice! How many hours per week can you dedicate to learning?"

await run_session(runner, "10 hours", "my-session")
# Intake Agent: [Calls validate_user_profile()] ✅ Profile complete!
# Research Agent: [Researches Data Science transition]
# Mentor Agent: [Creates personalized plan]